<a href="https://colab.research.google.com/github/luyandac35/mvelo/blob/master/copy%20financial%20data%20predicting%20model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ Google Colab-ready code
# Includes 30m trend confirmation, 5m execution, stop loss/take profit, signal strength

!pip install yfinance lightgbm plotly --quiet

import yfinance as yf
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
import plotly.graph_objects as go
import datetime
import pytz
from sklearn.metrics import mean_absolute_error

local_tz = pytz.timezone("Africa/Johannesburg")

# === Fetching Data ===
def fetch_data(ticker="GC=F", interval="5m", period="1d"):
    print(f"[{datetime.datetime.now(local_tz)}] Fetching {interval} data...")
    df = yf.download(ticker, interval=interval, period=period, progress=False)
    if df.empty:
        raise ValueError("No data fetched. Check ticker or connection.")
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    df = df[['Open', 'High', 'Low', 'Close']]
    df.index = df.index.tz_localize(None).tz_localize('UTC').tz_convert(local_tz)
    return df

# === Features ===
def create_features(df):
    df["Open_shift1"] = df["Open"].shift(1)
    df["Close_shift1"] = df["Close"].shift(1)
    df["Open_shift2"] = df["Open"].shift(2)
    df["Close_shift2"] = df["Close"].shift(2)
    df["Return1"] = df["Close"].pct_change()
    df["Volatility"] = df["High"] - df["Low"]
    return df.dropna()

# === Train Models ===
def train_models(df):
    feature_cols = ["Open_shift1", "Close_shift1", "Open_shift2", "Close_shift2", "Return1", "Volatility"]
    X = df[feature_cols]
    y_open = df["Open"]
    y_close = df["Close"]

    model_open = LGBMRegressor().fit(X, y_open)
    model_close = LGBMRegressor().fit(X, y_close)

    pred_close = model_close.predict(X)
    mae = mean_absolute_error(y_close, pred_close)

    return model_open, model_close, mae

# === Predict ===
def predict_next(model_open, model_close, df):
    feature_cols = ["Open_shift1", "Close_shift1", "Open_shift2", "Close_shift2", "Return1", "Volatility"]
    latest = df.iloc[-1:][feature_cols]
    pred_open = model_open.predict(latest)[0]
    pred_close = model_close.predict(latest)[0]
    return pred_open, pred_close

# === Signal ===
def generate_signal(current_price, predicted_price):
    if predicted_price > current_price * 1.001:
        return "BUY 🟢"
    elif predicted_price < current_price * 0.999:
        return "SELL 🔴"
    else:
        return "HOLD ⚪"

# === Risk (SL/TP) and Confidence ===
def calculate_risk(volatility, mae):
    stop_loss = volatility
    take_profit = 2 * volatility
    if mae < 2:
        confidence = "High"
    elif mae < 4:
        confidence = "Medium"
    else:
        confidence = "Low"
    return stop_loss, take_profit, confidence

# === Combined Run ===
def run_combined():
    df_30 = fetch_data(interval="30m", period="2d")
    df_30 = create_features(df_30)
    model_open_30, model_close_30, mae_30 = train_models(df_30)
    pred_open_30, pred_close_30 = predict_next(model_open_30, model_close_30, df_30)
    current_30 = df_30["Close"].iloc[-1]
    signal_30 = generate_signal(current_30, pred_close_30)

    df_5 = fetch_data(interval="5m", period="1d")
    df_5 = create_features(df_5)
    model_open_5, model_close_5, mae_5 = train_models(df_5)
    pred_open_5, pred_close_5 = predict_next(model_open_5, model_close_5, df_5)
    current_5 = df_5["Close"].iloc[-1]
    signal_5 = generate_signal(current_5, pred_close_5)

    volatility = df_5["Volatility"].rolling(10).mean().iloc[-1]
    sl, tp, confidence = calculate_risk(volatility, mae_5)

    print("\n===== Combined Trend Signal =====")
    print(f"📈 30-min Signal: {signal_30}  | 5-min Signal: {signal_5}  | Confidence: {confidence}")
    print(f"   Current Price: {current_5:.2f} USD")
    print(f"   Predicted Close: {pred_close_5:.2f} USD")
    print(f"   📉 Stop Loss:  {sl:.2f} USD")
    print(f"   📈 Take Profit: {tp:.2f} USD")

run_combined()


[2025-05-17 10:17:16.829449+02:00] Fetching 30m data...
YF.download() has changed argument auto_adjust default to True
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000196 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 162
[LightGBM] [Info] Number of data points in the train set: 78, number of used features: 6
[LightGBM] [Info] Start training from score 3200.064115
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warn